In [1]:
import pandas as pd
import numpy as np
import statistics


def median_of_ratios(data):
    d = pd.DataFrame(data.loc[data.min(axis=1) > 0])
    pseudo_column = []
    for gene, row in d.iterrows():
        pseudo_column.append(statistics.geometric_mean(row))
    d['pseudo'] = pseudo_column
    return d.apply(lambda x: x / d['pseudo']).drop('pseudo', axis=1).\
        apply(lambda x: x.median()).values


df = pd.read_csv('TCGA-COAD_cancer_normal.tsv', sep='\t', index_col=0)
gl = pd.read_csv("gene_lengths.tsv", sep="\t", index_col=0).sort_index()

size_factors = median_of_ratios(df)

RPM = df.div(df.sum(axis=0), axis=1) * 1e+6
DESeq2_RPM = RPM.div(size_factors, axis=1)
DESeq2_RPKM = DESeq2_RPM.div(gl["Length"], axis=0) * 1000
DESeq2_RPKM = np.log2(DESeq2_RPKM + 1)

# анализируем из 50% самых высокоэкспрессированных генов

median = DESeq2_RPKM.median(axis=1)
highly_expressed = median.sort_values(ascending=False).iloc[:len(median)//2]
DESeq2_RPKM = DESeq2_RPKM.loc[highly_expressed.index]

mean_expression_dif = abs(DESeq2_RPKM.iloc[:, :5].mean(axis=1)
                          - DESeq2_RPKM.iloc[:, 5:].mean(axis=1))
highest_dif = mean_expression_dif.sort_values(ascending=False).iloc[:10]
print(f'The genes with highest difference in average expression between normal '
      f'and cancer: {", ".join(list(highest_dif.index))}')

The genes with highest difference in average expression between normal and cancer: DPEP1, CLCA4, CA1, CD177, GUCA2A, AQP8, MS4A12, CEACAM7, SPP1, ZG16
